## Preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving NIJ_s_Recidivism_Challenge_Full_Dataset.csv to NIJ_s_Recidivism_Challenge_Full_Dataset.csv


In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from google.colab import files
import pandas as pd
import tensorflow as tf
import keras
import warnings
import pandas as pd
import time
start_time = time.time()
df = pd.read_csv('/content/recidivism.csv')
df.head()

,ID,Gender,Race,Age_at_Release,Residence_PUMA,Gang_Affiliated,Supervision_Risk_Score_First,Supervision_Level_First,Education_Level,Dependents,...,DrugTests_Meth_Positive,DrugTests_Other_Positive,Percent_Days_Employed,Jobs_Per_Year,Employment_Exempt,Recidivism_Within_3years,Recidivism_Arrest_Year1,Recidivism_Arrest_Year2,Recidivism_Arrest_Year3,Training_Sample
0,1,M,BLACK,43-47,16,False,3.0,Standard,At least some college,3 or more,...,0.000000,0.0,0.488562,0.447610,False,False,False,False,False,1
1,2,M,BLACK,33-37,16,False,6.0,Specialized,Less than HS diploma,1,...,0.000000,0.0,0.425234,2.000000,False,True,False,False,True,1
2,3,M,BLACK,48 or older,24,False,7.0,High,At least some college,3 or more,...,0.166667,0.0,0.000000,0.000000,False,True,False,True,False,1
3,4,M,WHITE,38-42,16,False,7.0,High,Less than HS diploma,1,...,0.000000,0.0,1.000000,0.718996,False,False,False,False,False,1
4,5,M,WHITE,33-37,16,False,4.0,Specialized,Less than HS diploma,3 or more,...,0.058824,0.0,0.203562,0.929389,False,True,True,False,False,1


In [ ]:
# Determine the number of unique values in each column.
df.nunique()

ID                                                   25835
Gender                                                   2
Race                                                     2
Age_at_Release                                           7
Residence_PUMA                                          25
Gang_Affiliated                                          2
Supervision_Risk_Score_First                            10
Supervision_Level_First                                  3
Education_Level                                          3
Dependents                                               4
Prison_Offense                                           5
Prison_Years                                             4
Prior_Arrest_Episodes_Felony                            11
Prior_Arrest_Episodes_Misd                               7
Prior_Arrest_Episodes_Violent                            4
Prior_Arrest_Episodes_Property                           6
Prior_Arrest_Episodes_Drug                              

In [ ]:
# convert categorical variable into dummy/indicator variables
dummy_df = pd.get_dummies(df)

In [ ]:
# Split our preprocessed data into our features and target arrays
X = dummy_df.drop('Recidivism_Within_3years', axis = 1)
y = dummy_df['Recidivism_Within_3years']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

In [ ]:
# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

In [ ]:
# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])
first_hidden_layer = 10
second_hidden_layer = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=first_hidden_layer, input_dim=input_features, activation="tanh"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=second_hidden_layer, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Disable the SettingWithCopyWarning
pd.options.mode.chained_assignment = None

# Train the neural network model
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    nn.fit(X_train_scaled, y_train, epochs=10, verbose=0)

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=0)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Disable the SettingWithCopyWarning
pd.options.mode.chained_assignment = None

# Define the filename
# iterations = 3
filename = f"recidivism.h5"

# Save the neural network model with the constructed filename
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    nn.save(filename)

# Download the file
files.download(filename)

# Direct the user where to find the file
print(f"{filename} has been exported to your Downloads folder.")
print(time.time()-start_time)